In [1]:
import pandas as pd

In [5]:
df_chicago=pd.read_excel("cta-ridership-daily-boarding-totals.xlsx")


In [ ]:
df_chicago.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343


In [10]:
df_chicago.shape

(9100, 5)

In [11]:
print("Description de la DataFrame Mode: \n")
df_chicago.describe()

Description de la DataFrame Mode: 



,service_date,bus,rail_boardings,total_rides
count,9100,9.100000e+03,9.100000e+03,9.100000e+03
mean,2013-06-16 12:01:25.846153984,7.044593e+05,5.064497e+05,1.210909e+06
min,2001-01-01 00:00:00,8.078300e+04,2.354400e+04,1.100470e+05
25%,2007-03-25 18:00:00,4.607805e+05,3.265512e+05,7.975130e+05
50%,2013-06-16 12:00:00,7.505730e+05,5.718135e+05,1.399761e+06
75%,2019-09-08 06:00:00,9.553080e+05,6.876832e+05,1.626020e+06
max,2025-11-30 00:00:00,1.211992e+06,1.146516e+06,2.049519e+06
std,NaN,2.691563e+05,2.067194e+05,4.629037e+05


In [12]:
print("Info sur la DataFrame Mode \n")
df_chicago.info()

Info sur la DataFrame Mode 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    9100 non-null   datetime64[ns]
 1   day_type        9100 non-null   object        
 2   bus             9100 non-null   int64         
 3   rail_boardings  9100 non-null   int64         
 4   total_rides     9100 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 355.6+ KB


In [13]:
print("Les valeurs nulles dans le DataFrame Route :")
df_chicago.isna().sum()

Les valeurs nulles dans le DataFrame Route :


service_date      0
day_type          0
bus               0
rail_boardings    0
total_rides       0
dtype: int64

In [14]:

print("Les doublons dans le DataFrame Mode:",df_chicago.duplicated().sum())

Les doublons dans le DataFrame Mode: 0


Nettoyage: Filtrer la date de 2019 à 2025

In [15]:
df_chicago=df_chicago[(df_chicago['service_date']> '2018-12-31')]

Exportation du DataFrame final au format CSV

In [16]:
df_chicago.to_csv("../processed/Chicago_mode.csv")

Fichier RDF
EDA

In [18]:
from rdflib import Graph
import pandas as pd
import os
# RDF => DataFrame

In [47]:
# daily RDF to DataFrame
def rdf_daily_to_df(file_path):
    g = Graph()
    g.parse(file_path, format="xml")

    rows = {}

    for s, p, o in g:
        s = str(s)
        p = str(p)
        o = str(o)

        if s not in rows:
            rows[s] = {
                "subject": s,
                "city": "Chicago"
            }

        col = p.split("/")[-1]

        if col == "route":
            rows[s]["route"] = o
        elif col == "date":
            rows[s]["date"] = o
        elif col == "daytype":
            rows[s]["daytype"] = o
        elif col == "rides":
            rows[s]["ridership"] = o

    df = pd.DataFrame(rows.values())

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["ridership"] = pd.to_numeric(df["ridership"], errors="coerce")

    return df

dfs = []

rdf_dir = "rdf_CTA__Ridership__Daily_by_Route_routes_2001_2025"

for file in os.listdir(rdf_dir):
    if file.endswith(".rdf"):
        full_path = os.path.join(rdf_dir, file)
        print("Reading:", file)
        df_temp = rdf_daily_to_df(full_path)
        dfs.append(df_temp)

df_Daily = pd.concat(dfs, ignore_index=True)




Reading: rdf_CTA__Ridership__Daily_by_Route_routes_1.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_10.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_11.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_12.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_13.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_14.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_15.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_16.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_17.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_18.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_19.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_2.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_3.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_4.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_5.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_6.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_7.rdf
Reading: rdf_CTA__Rid

In [48]:
df_Daily.head()

,subject,city,date,route,daytype,ridership
0,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,2019-11-08,100,W,489
1,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,2002-05-27,10,U,1122
2,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,2015-10-26,1,W,1945
3,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,2003-05-01,106,W,2683
4,https://data.cityofchicago.org/resource/jyb9-n...,Chicago,2025-04-07,106,W,725


In [49]:
print("Description de la DataFrame Daily \n",df_Daily.describe())

Description de la DataFrame Daily 
                                 date     ridership
count                        1092474  1.092474e+06
mean   2013-03-20 02:06:30.568928768  5.833470e+03
min              2001-01-01 00:00:00  0.000000e+00
25%              2007-02-02 00:00:00  1.269000e+03
50%              2012-12-08 00:00:00  3.674000e+03
75%              2019-05-18 00:00:00  8.669000e+03
max              2025-11-30 00:00:00  4.517700e+04
std                              NaN  6.037996e+03


In [50]:
print("Info sur la DataFrame Daily \n")
df_Daily.info()

Info sur la DataFrame Daily 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092474 entries, 0 to 1092473
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   subject    1092474 non-null  object        
 1   city       1092474 non-null  object        
 2   date       1092474 non-null  datetime64[ns]
 3   route      1092474 non-null  object        
 4   daytype    1092474 non-null  object        
 5   ridership  1092474 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 50.0+ MB


In [51]:

print("Daily DataFrame shape:", df_Daily.shape)

Daily DataFrame shape: (1092474, 6)


In [52]:
print("Les valeurs nulles dans le DataFrame Daily :\n", df_Daily.isnull().sum())

Les valeurs nulles dans le DataFrame Daily :
 subject      0
city         0
date         0
route        0
daytype      0
ridership    0
dtype: int64


In [53]:
print("Les doublons dans le DataFrame Daily:",df_Daily.duplicated().sum())

Les doublons dans le DataFrame Daily: 0


NETTOYAGE

In [55]:
# Filtrer des dates
df_Daily=df_Daily[(df_Daily['date']> '2018-12-31')]

In [56]:
#supprimer colonnes subject et city des deux DataFrames
df_Daily = df_Daily.drop(columns=["subject", "city"])


In [58]:
df_Daily.to_csv("../processed/Chicago_route.csv", index=False)